### Vì tựa game này là 1 tựa game khá là phức tạp và có nhiều cơ chế nên ta cần tìm hiểu chúng và 1 trong những điều quan trọng nhất chính là các vị tướng, vậy bạn có bao giờ tự hỏi các vị tướng có vai trò thế nào trong đội hình chưa ? Phần sau đây sẽ cho ta biết rõ thêm về các vị tướng để mọi người có thể dễ dàng nắm bắt hơn.

Ta sẽ thực hiện gom cụm các vị tướng để có thể biết được các vị tướng thuộc nhóm nào trong game để dễ dàng sắp xếp và lên đồ phù hợp cho chúng.

In [1]:
import pandas as pd
import numpy as np
from itertools import chain
import ast
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.metrics import silhouette_score

Đọc dữ liệu từ file:

In [2]:
item_df = pd.read_csv('../Data/process_data_tft_each_item_csv.csv', usecols=[
                      'ID Match', 'Champion', 'List Name Item']).drop_duplicates().reset_index(drop=True)
item_df = item_df[['Champion', 'List Name Item']]
item_df

,Champion,List Name Item
0,"['TFT8_Gangplank', 'TFT8_Vi', 'TFT8_LeeSin', '...","[['TFT_Item_ZekesHerald'], ['TFT_Item_BrambleV..."
1,"['TFT8_Lux', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT8...","[['TFT8_Item_HeartEmblemItem'], ['TFT_Item_Bra..."
2,"['TFT8_Galio', 'TFT8_Nasus', 'TFT8_Gangplank',...","[['TFT_Item_GargoyleStoneplate'], [], ['TFT_It..."
3,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[[], ['TFT8_Item_HackerEmblemItem'], ['TFT_Ite..."
4,"['TFT8_Lulu', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT...","[[], ['TFT_Item_WarmogsArmor', 'TFT_Item_Titan..."
...,...,...
18091,"['TFT8_Blitzcrank', 'TFT8_Renekton', 'TFT8_Vi'...","[[], ['TFT_Item_ArchangelsStaff', 'TFT_Item_Je..."
18092,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[['TFT_Item_TitanicHydra'], ['TFT_Item_ChainVe..."
18093,"['TFT8_WuKong', 'TFT8_Jax', 'TFT8_Zoe', 'TFT8_...","[[], [], ['TFT8_Item_HeartEmblemItem'], ['TFT_..."
18094,"['TFT8_Kayle', 'TFT8_Gangplank', 'TFT8_Fiora',...","[[], [], [], [], ['TFT_Item_StatikkShiv', 'TFT..."


Đổi kiểu dữ liệu từ String sang List

In [3]:
def changeStringToList(col_name, df):
    for i in range(len(col_name)):
        temp = []
        for j in range(len(df[col_name[i]])):
            temp.append(ast.literal_eval(df[col_name[i]][j]))
        df[col_name[i]] = temp

In [4]:
col_name_type_list = ['Champion', 'List Name Item']
changeStringToList(col_name_type_list, item_df)

Tạo dataframe với mỗi tướng sẽ là trang bị gì:

In [5]:
list_champion = list(chain.from_iterable(item_df['Champion']))
list_item_of_champion = list(chain.from_iterable((item_df['List Name Item'])))

champion_df = pd.DataFrame(data=list_champion, columns=['Champion'])
item_of_champions_df = pd.DataFrame(data= list_item_of_champion, columns=['First', 'Second', 'Third', '0', '1', '2', '3']).iloc[:,:-4]

extracted_col = champion_df['Champion']

item_of_champions_df.insert(0, "Champion", extracted_col)
pos = ['First', 'Second', 'Third']
list_champion_unique = sorted(champion_df['Champion'].unique())


Với mỗi tướng sẽ tạo ra tập các trang bị được sử dụng với tỉ lệ là bao nhiêu:

In [6]:
list_item_unique = list(set(i for j in list_item_of_champion for i in j))
list_item_unique
data_model = pd.DataFrame(columns=list_champion_unique, index=list_item_unique)
for champ in list_champion_unique:
    first_item = item_of_champions_df[['Champion','First']].value_counts()[champ]
    second_item = item_of_champions_df[['Champion','Second']].value_counts()[champ]
    third_item = item_of_champions_df[['Champion','Third']].value_counts()[champ]  
    freq = first_item.add(second_item.add(third_item, fill_value=0), fill_value=0).sort_values(ascending=False) / first_item.add(second_item.add(third_item, fill_value=0), fill_value=0).sort_values(ascending=False).sum()
    data_model[champ] = freq

Chuyển đổi giá trị Nan thành 0 (ứng với những trang bị không được lên)

In [7]:
data_model = data_model.fillna(0).T
data_model

,TFT5_Item_GiantSlayerRadiant,TFT5_Item_WarmogsArmorRadiant,TFT8_Item_Warmogs_GenAE,TFT5_Item_SunfireCapeRadiant,TFT_Item_NeedlesslyLargeRod,TFT5_Item_GargoyleStoneplateRadiant,TFT8_Item_RenegadeEmblemItem,TFT5_Item_RunaansHurricaneRadiant,TFT5_Item_RabadonsDeathcapRadiant,TFT_Item_EmptyBag,...,TFT8_Item_IonicSpark_GenAE,TFT4_Item_OrnnRanduinsSanctum,TFT8_Item_HandOfJustice_GenAE,TFT8_Item_DuelistEmblemItem,TFT_Item_LocketOfTheIronSolari,TFT8_Item_PranksterEmblemItem,TFT8_Item_InterPolarisEmblemItem,TFT5_Item_RapidFirecannonRadiant,TFT_Item_IonicSpark,TFT5_Item_BloodthirsterRadiant
TFT8_Alistar,0.000000,0.000238,0.005474,0.000238,0.002380,0.000000,0.002380,0.000238,0.000000,0.002380,...,0.004046,0.002380,0.000000,0.003332,0.028082,0.019990,0.001190,0.000000,0.052594,0.000000
TFT8_Annie,0.000000,0.000629,0.006446,0.001100,0.000943,0.000629,0.000786,0.000000,0.000000,0.000314,...,0.007546,0.000943,0.000943,0.000157,0.006288,0.004873,0.000000,0.000000,0.132212,0.000000
TFT8_Aphelios,0.001090,0.000000,0.000000,0.000000,0.001907,0.000000,0.019074,0.000545,0.000000,0.005995,...,0.000000,0.000272,0.000000,0.020163,0.001362,0.001090,0.003815,0.000272,0.001362,0.000000
TFT8_Ashe,0.000909,0.000000,0.000000,0.000000,0.000909,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.041818,0.000000,0.000909,0.000000,0.000000,0.000000,0.000000
TFT8_AurelionSol,0.000825,0.000000,0.000000,0.000000,0.008663,0.000000,0.003300,0.000000,0.000413,0.000000,...,0.000000,0.000000,0.000000,0.002888,0.000825,0.000825,0.000413,0.000000,0.001238,0.000000
TFT8_BelVeth,0.001334,0.000000,0.000000,0.000000,0.001334,0.000445,0.005336,0.000889,0.000000,0.000889,...,0.000000,0.000445,0.000000,0.016896,0.001334,0.000889,0.001779,0.000000,0.002223,0.000889
TFT8_Blitzcrank,0.000000,0.000000,0.000000,0.001889,0.000000,0.000000,0.007554,0.000000,0.000000,0.000000,...,0.000000,0.002833,0.000000,0.000944,0.022663,0.000000,0.002833,0.000000,0.063267,0.000000
TFT8_Camille,0.000568,0.000000,0.000568,0.000000,0.001135,0.000000,0.000000,0.000568,0.000000,0.003405,...,0.000000,0.000000,0.000000,0.002270,0.003973,0.000000,0.000000,0.000000,0.005108,0.000000
TFT8_Chogath,0.000000,0.000974,0.000000,0.000000,0.003408,0.000000,0.000487,0.000000,0.000000,0.004869,...,0.000487,0.000974,0.000000,0.000974,0.015579,0.002434,0.000487,0.000000,0.079357,0.000000
TFT8_Draven,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001351,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.005405,0.000000,0.000000,0.000000,0.001351,0.000000,0.000000


In [8]:
# print(data_model.loc['TFT8_Lux'].sort_values(ascending=False)[:10])

Lưu dữ liệu vào file csv

In [9]:
# data_model.to_csv('data_model_csv.csv')

Sử dụng dữ liệu được tạo ra phía trên để thực hiện gom cụm bằng thuật toán của thư viện sklearn với số cụm là 4 tương ứng với 4 nhóm tướng chính của trò chơi này:

In [10]:
clustering = AgglomerativeClustering(n_clusters=4).fit(data_model)
clustering.labels_

array([3, 3, 0, 0, 1, 0, 3, 0, 3, 0, 3, 1, 2, 2, 3, 2, 2, 0, 2, 2, 0, 2,
       2, 3, 1, 2, 3, 1, 2, 2, 3, 2, 3, 3, 3, 3, 3, 0, 3, 0, 3, 0, 1, 1,
       3, 2, 1, 2, 2, 0, 1, 3, 2, 2, 2, 1, 3, 0, 1], dtype=int64)

Sau khi gom cụm xong ta phân được 4 nhóm khác nhau, bây giờ ta gắn nó với tên nhóm tương ứng:

In [11]:
champ_type = ['dame', 'magic','utility','tank']
for i in range(len(data_model.index)):
    print(data_model.index[i], ': ', champ_type[clustering.labels_[i]])

TFT8_Alistar :  tank
TFT8_Annie :  tank
TFT8_Aphelios :  dame
TFT8_Ashe :  dame
TFT8_AurelionSol :  magic
TFT8_BelVeth :  dame
TFT8_Blitzcrank :  tank
TFT8_Camille :  dame
TFT8_Chogath :  tank
TFT8_Draven :  dame
TFT8_Ekko :  tank
TFT8_Ezreal :  magic
TFT8_Fiddlesticks :  utility
TFT8_Fiora :  utility
TFT8_Galio :  tank
TFT8_Gangplank :  utility
TFT8_Janna :  utility
TFT8_Jax :  dame
TFT8_Jinx :  utility
TFT8_Kaisa :  utility
TFT8_Kayle :  dame
TFT8_Leblanc :  utility
TFT8_LeeSin :  utility
TFT8_Leona :  tank
TFT8_Lulu :  magic
TFT8_Lux :  utility
TFT8_Malphite :  tank
TFT8_MissFortune :  magic
TFT8_Mordekaiser :  utility
TFT8_Nasus :  utility
TFT8_Nilah :  tank
TFT8_Nunu :  utility
TFT8_Poppy :  tank
TFT8_Rammus :  tank
TFT8_Rell :  tank
TFT8_Renekton :  tank
TFT8_Riven :  tank
TFT8_Samira :  dame
TFT8_Sejuani :  tank
TFT8_Senna :  dame
TFT8_Sett :  tank
TFT8_Sivir :  dame
TFT8_Sona :  magic
TFT8_Soraka :  magic
TFT8_Sylas :  tank
TFT8_Syndra :  utility
TFT8_Taliyah :  magic
TFT8_Talo

##### Thế là ta đã có được từng vị tướng sẽ có vai trò gì trong đội hình để dễ dàng cho việc sắp xếp (ví dụ nhưng tướng tank đứng trước tướng dame, phép đứng sau) và lên đồ cho chúng như thế nào (tank thì lên các món tăng chống chịu, pháp sư thì lên đồ phép còn dame thì lên đồ sát thương vật lý). Những người chơi mới có thể dựa vào đây mà làm quen trò chơi này nhanh hơn và cảm thấy thú vị hơn.